### Namespaces and variable scoping

*One of the most tricky aspects of Python*...

In [1]:
a=1

def my_func():
    a=20
    print("\nPrint of 'a' within the function: ",a)
    
print("The variable 'a' defined outside the function\nbefore the function is invoked: ", a)    

my_func()

print("\nThe variable after the function is invoked: ", a)

The variable 'a' defined outside the function
before the function is invoked:  1

Print of 'a' within the function:  20

The variable after the function is invoked:  1


The variable *a* defined inside the function *my_func* is a ***local*** variable: it occupies a specific position in memory which is reserved for the objects *belonging* to the function. Such position is *different* from the one reserved to the homonymous variable defined in the *main module*. The two variables just happen to share a common label (*a*) but point at two different addresses in the computer memory.

Another feature of variables is illustrated here:

In [2]:
a=1

def my_func():
    b=20
    print("\nPrint of 'a' within the function: ",a)
    print("Print of 'b' within the function: ", b)
    
print("The variable 'a' defined outside the function\nbefore the function is invoked: ", a)    

my_func()

The variable 'a' defined outside the function
before the function is invoked:  1

Print of 'a' within the function:  1
Print of 'b' within the function:  20


That is, the variable *a* is *seen* inside the function *my_func*: at variance with the previous definition of such function, here, the ***global*** variable *a* is not *shadowed*, inside the function, by a local variable having the same label. 

Now if we ask for the variable *a*, we get:

In [3]:
print(a)

1


... but, if we ask for the local variable *b* defined in *my_func* we get:

In [4]:
print(b)

NameError: name 'b' is not defined

... that is, an ***error***: the local variable *b* no longer exists, as after the function is executed, the memory is cleared.

Now, we see another example (but, before of that, we free the *global* memory from the variable *a*...)

In [5]:
if 'a' in globals(): del(a)

Now, we define a *nested* function (a function *inside* a function):

In [6]:
def func_1():
    a=1
    print("1) ", a)

    def func_2():
        a=20
        print("2) ", a)
    
    func_2()
    
    print("3) ", a)

then we call *func_1*:

In [7]:
func_1()

1)  1
2)  20
3)  1


That is: 

- the variable *a* defined in *func_1* is *seen* in the function *func_2* which, in turn, is defined *inside* *func_1* (*func_2* is a nested function); 
- *func_2* shadows the original value of *a* by assigning its local *a*=20; 
- after *func_2* ends, its *namespace* is cleared, and *func_1* prints its original *a* value (1).

The value *a=1* is however *local* within *func_1*; indeed, if we ask for *a*, now, we get:

In [8]:
print(a)

NameError: name 'a' is not defined

that is, our *usual* error.

### Take home message:
- Variables defined in the *main* module are *global*: they can be seen in every function defined in the *same* module, unless they are *shadowed* by variables, defined inside the function, that share the same label. 
- Variables defined inside a function can be seen in all the nested (sub)functions defined in that function (unless *shadowed*).

#### Warning 

Have a look here at the (*wrong*) code below: 

In [9]:
def no_good():
    counter=1
    print("Counter ", counter)
    def nested_func():
        counter=counter+1
        print("Counter ", counter)
    
    nested_func()

No problem at the time of the function definition, but...

In [10]:
no_good()

Counter  1


UnboundLocalError: cannot access local variable 'counter' where it is not associated with a value

We got a *UnboundLocalError for 'counter' which is not associated with a value*. 

This error is generated when *nested_func* is invoked within *no_good*... Why? If *counter* is defined outside *nested_func*, it should be seen within it! 

Well, not quite... The point is that *nested_func* tries to reassign the variable *counter*:

```
counter=counter+1
```

and, at that very moment, *counter* becomes a local variable shadowing the one defined in the outer *namespace*, but then Python does no longer know its value when trying to evaluate the right hand side of the expression pretending to (re)compute the variable...

This other function does not show any problem, but probably it does not exactly do what we would like it should...

In [11]:
def is_good():
    counter=1
    print("Counter    ", counter)
    def nested_func():
        counter_2=counter+1               # this is another counter...
        print("Counter(2) ", counter_2)
    
    nested_func()
    nested_func()

In [12]:
is_good()

Counter     1
Counter(2)  2
Counter(2)  2


Here is a possible way to *bypass* the problem: the variable *counter* is *declared* as **global** in all the functions and nested functions that modify it:

In [13]:
if 'counter' in globals(): del(counter)  # clear the memory from 'counter' if it is defined

def is_good():
    global counter                        # counter is declared as global
    counter=1
    print("Counter ", counter)
    def nested_func():
        global counter                    # counter is again declared as global
        counter=counter+1
        print("Counter ", counter)
    
    nested_func()
    nested_func()
    
    counter=counter+1
    print("Counter ",counter)

In [14]:
is_good()

Counter  1
Counter  2
Counter  3
Counter  4


In fact:

- the first ``` global counter ``` declaration, and the subsequent assignement ``` counter = 1 ```, does create a *bind* between the *counter* variable and its value at the *global* level (that is, in the *namespace* of the *main* module); 
- the second ``` global counter ``` declaration (in the nested function) does create a bind between the variable *counter* used in the nested function and the one already defined at the global level (again in the *main namespace*)

The function works well, but the side effect is that, now, *counter* is not cleared when *is_good* is terminated...

In [15]:
print(counter)

4


In the same way (*counter* defined in the main namespace):

In [16]:
if 'counter' in globals(): del(counter)
    
counter=0

def func():
    global counter
    counter=counter+1
    print(counter)
    
func()   

1


... and now, *counter* is *permanently* modified:

In [17]:
print(counter)

1


This *bypass* of the *global* declaration inside functions might seem to be very useful, but is absolutely **not *encouraged*** as it is at the origin of uncountable errors and bugs in Python codes. Therefore, use it only if you cannot do otherwise (*and you always can do otherwise*... we will see how).

Indeed, try to imagine a large program that *somewhere* defines and assigns, at the global level, the variable *counter*, having some meaning, being used in some operations. Now, *unaware* of all that, you takes that program and modify it by *just* adding a function that uses a variable you give the very same name *counter* which is declared as *global*... 

To limit *damages* like those, you can use ***nonlocal*** declarations!

### *nonlocal* declaration

Here is another declaration (***nonlocal***) that is particularly useful in case of nested functions:

In [18]:
if 'counter' in globals(): del(counter)
    
def func():
    counter=1
    def nested_func():
        nonlocal counter
        counter=counter+1
        
    print("Before the call to nested_func: ", counter)
    nested_func()
    print("After the call to nested_func:  ", counter)

We define a variable *counter* within a function (*func*); then, within the nested function *nested_func* we reassign *counter* (through ``` counter = counter + 1 ```) but declaring the variable as *nonlocal*: this creates a binding of the latter variable with the one defined within the function *func*; it is not a binding at the global level, in the namespace of the module (the *cell*), but it is in the namespace of the function *func*. Let's call *func*: 

In [19]:
func()

Before the call to nested_func:  1
After the call to nested_func:   2


The *proof* that *counter* is not defined at the global level is:

In [20]:
print('counter' in globals())

False


In [21]:
counter=0
func()
print("Original counter: ", counter)

Before the call to nested_func:  1
After the call to nested_func:   2
Original counter:  0


### Exercise:

Use the *nonlocal* declaration to code a function for the calculation of the factorial of a number and *recursion* (in a previous lecture we used *classes*, not yet explained, to code the algorithm).

In [22]:
def factorial(n):
    fact=1
    
    def rec_func():
        nonlocal fact, n
        fact=fact*n
        n=n-1
        if n == 1:
            return
        else:
            rec_func()
            
    rec_func()
    return fact       

Here we created a bind of the *fact* variable, in the *rec_func*, with the variable having the same name in the namespace of the function *factorial*. The same is true for the variable *n* which is an argument of the function *factorial* and, as such, it exists in the namespace of the latter function; we need this *nonlocal* declaration as, as well as for *fact*, *n* is reassigned within *rec_func* and we want to avoid shadowing. 

Let's see if it works...

In [23]:
factorial(5)

120

### Exercise: square root of a number

Use the recursion method to compute the square root of a number $n$, by the Newton's algorithm:


$\sqrt{n} = \lim\limits_{m \to \infty} x_m$ 

where

$x_{i+1}=\frac{1}{2}\left(x_i+\frac{n}{x_i}\right)$ and $x_0$ is an initial *guess*


    

In [24]:
def square_root(n, thr=1e-15):
    '''
    Computes the square root of a number within a given precision
    
    Args:
        n:   number whose square root has to be computed
        thr: precision required
    '''  
    guess=n
    if guess == 0.: guess=1.
        
    def square_rec():
        nonlocal guess
        approx=0.5*(guess+n/guess)
        diff=abs(approx-guess)
        if diff > thr:
           guess=approx
           square_rec()
        else:
           return
    
    square_rec()
    return guess

Check if this function works...

In [25]:
square_root(2)

1.414213562373095

Let's see the difference between the square root of 2 computed by using the *numpy* *sqrt* function and that computed by our recursive function:

In [26]:
import numpy as np
np.sqrt(2)-square_root(2)

np.float64(2.220446049250313e-16)

As expected, the difference is of the order of magnitude specified by the threshold *thr* variable

### Lists and arrays are *apparently* different...

To further complicate the issue!

*Apparently*, lists do behave in the same way as scalar variables:

In [27]:
a=[1,2,3,4,5]

def func():
    a=[6,7,8]
    print("a within func:  ", a)
    
func()
print("a outside func: ", a)

a within func:   [6, 7, 8]
a outside func:  [1, 2, 3, 4, 5]


Up to now, nothing strange: just the usual shadowing... Now, look at here:

In [28]:
a=[1,2,3,4,5]

def func():
    a[0]=7
    print("a within func:      ", a)
    
print("a before func call: ", a)
func()
print("a after func call:  ", a)

a before func call:  [1, 2, 3, 4, 5]
a within func:       [7, 2, 3, 4, 5]
a after func call:   [7, 2, 3, 4, 5]


... but, wait a second... the list has been modified even if there is no *global* declaration of it inside the function... 

So, what? Are *lists* always globals? Not at all... Let's try this:

In [29]:
a=[1,2,3,4,5]

def func():
    a=a*2
    print("a within func:      ", a)
    
print("a before func call: ", a)
func()
print("a after func call:  ", a)

a before func call:  [1, 2, 3, 4, 5]


UnboundLocalError: cannot access local variable 'a' where it is not associated with a value

... Indeed, inside the function *func*, the reassigment of the list *a* causes it to became *local*, exactly as we already saw for scalars. Therefore, the right hand side of ``` a=a*2 ``` cannot be evaluated (after all, what's the value of this *new* local *a*?). The bypass is the same as for scalar: use *global*:

In [30]:
a=[1,2,3,4,5]

def func():
    global a
    a=a*2
    print("a within func:      ", a)
    
print("a before func call: ", a)
func()
print("a after func call:  ", a)

a before func call:  [1, 2, 3, 4, 5]
a within func:       [1, 2, 3, 4, 5, 1, 2, 3, 4, 5]
a after func call:   [1, 2, 3, 4, 5, 1, 2, 3, 4, 5]


Since *a* is a Python list, ``` a*2 ``` is just the *concatenation* of two copies of the list, and it is *permanently* modified. 

The label *a* of the list points at some location in memory where its elements are stored. If the list is reassigned within a function, it points to another location in memory and so, definitely, it is a new list (unless declared as *global*) that shadows the original one. However, ***elements of the list maintain the same addresses in memory***, so that they can be modified... 

As a further example:

In [31]:
a=[1,2,3,4,5]

def func(factor):
    for ia in range(len(a)):
        a[ia]=a[ia]*factor

In [32]:
print(a)
func(4)
print(a)

[1, 2, 3, 4, 5]
[4, 8, 12, 16, 20]


Same behaviour with numpy arrays as elements are addressed: 

In [33]:
import numpy as np

a=np.array([1,2,3,4,5])

def func(factor):
    for ia in range(len(a)):
        a[ia]=a[ia]*factor
        
print(a)
func(4)
print(a)

[1 2 3 4 5]
[ 4  8 12 16 20]


On the other hand, an error is returned if the whole list is addressed (no *global* declaration)

In [34]:
import numpy as np

a=np.array([1,2,3,4,5])

def func(factor):
    a=a*factor
        
print(a)
func(4)
print(a)

[1 2 3 4 5]


UnboundLocalError: cannot access local variable 'a' where it is not associated with a value

Correct behaviour if the whole list is addressed and declared as *global*

In [35]:
import numpy as np

a=np.array([1,2,3,4,5])

def func(factor):
    global a
    a=a*factor
        
print(a)
func(4)
print(a)

[1 2 3 4 5]
[ 4  8 12 16 20]
